In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import re

# Get current notebook directory
current_dir = os.getcwd()

# Add parent directory
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import peptide
import math
import interpreter_modify

In [13]:
amino_acid_masses = {
        "A": 71.03711,   # Alanine
        "R": 156.10111,  # Arginine
        "N": 114.04293,  # Asparagine
        "D": 115.02694,  # Aspartic acid
        "C": 103.00919,  # Cysteine
        "E": 129.04259,  # Glutamic acid
        "Q": 128.05858,  # Glutamine
        "G": 57.02146,   # Glycine
        "H": 137.05891,  # Histidine
        "I": 113.08406,  # Isoleucine
        "L": 113.08406,  # Leucine
        "K": 128.09496,  # Lysine
        "M": 131.04049,  # Methionine
        "F": 147.06841,  # Phenylalanine
        "P": 97.05276,   # Proline
        "S": 87.03203,   # Serine
        "T": 101.04768,  # Threonine
        "W": 186.07931,  # Tryptophan
        "Y": 163.06333,  # Tyrosine
        "V": 99.06841    # Valine
    }

In [2]:
def is_close(num1, num2, threshold = 0.01):
    return abs(num1-num2) <= threshold

In [5]:
import itertools

def construct_graph_nodes(spectrum):
    """
    Generates a list of nodes (x1, x2) from the input spectrum.
    
    Logic:
    1. Create a grid of |Spectrum| x |Spectrum|.
    2. Remove diagonal nodes where x1 == x2.
    3. Exception: Always keep the source node (0, 0).
    
    Args:
        spectrum (list of float): The list of masses.
        
    Returns:
        list of tuple: A list of (x1, x2) ordered pairs.
    """
    # Create the full Cartesian product (the grid)
    full_grid = itertools.product(spectrum, repeat=2)
    
    # Filter based on your rules
    nodes = [
        (x1, x2) for x1, x2 in full_grid
        if x1 != x2 or (x1 == 0 and is_close(x2, 18.01056))
    ]
    
    return nodes

# --- Example Usage ---
mass_spectrum = [0.0, 18.01056, 150.5, 300.2]
graph_nodes = construct_graph_nodes(mass_spectrum)

print(f"Input Spectrum: {mass_spectrum}")
print(f"Generated Nodes: {graph_nodes}")

Input Spectrum: [0.0, 18.01056, 150.5, 300.2]
Generated Nodes: [(0.0, 18.01056), (0.0, 150.5), (0.0, 300.2), (18.01056, 0.0), (18.01056, 150.5), (18.01056, 300.2), (150.5, 0.0), (150.5, 18.01056), (150.5, 300.2), (300.2, 0.0), (300.2, 18.01056), (300.2, 150.5)]


In [10]:
import math

from bisect import bisect_left

from bisect import bisect_left

def build_adjacency_graph(nodes, mass_list, threshold=0.01):
    """
    Constructs an adjacency dictionary graph using a list of allowed mass deltas.
    
    Args:
        nodes (list of tuple): List of valid (x1, x2) nodes.
        mass_list (list of float): List of allowed mass differences (AAs, PTMs, combinations).
        threshold (float): Tolerance for mass matching.
        
    Returns:
        dict: A dictionary where keys are (x1, x2) nodes and 
              values are lists of edges: [(target_node, mass_delta, type), ...]
    """
    # 1. Initialize the graph
    graph = {node: [] for node in nodes}
    
    # 2. Optimize Lookups
    valid_nodes_set = set(nodes)
    unique_masses = sorted(list(set([n[0] for n in nodes] + [n[1] for n in nodes])))
    
    def find_mass(target):
        """Binary search to find if target exists within threshold."""
        idx = bisect_left(unique_masses, target)
        if idx < len(unique_masses) and abs(unique_masses[idx] - target) <= threshold:
            return unique_masses[idx]
        if idx > 0 and abs(unique_masses[idx - 1] - target) <= threshold:
            return unique_masses[idx - 1]
        return None

    # 3. Iterate through every node in the graph
    for current_node in graph:
        x1, x2 = current_node
        max_current = max(x1, x2)
        
        # Iterate through the simple list of masses
        for mass in mass_list:
            
            # --- Check DOWN Edge: (x1, x2) -> (x1 + mass, x2) ---
            target_mass_x1 = find_mass(x1 + mass)
            
            if target_mass_x1 is not None:
                target_node = (target_mass_x1, x2)
                
                if target_node in valid_nodes_set:
                    # Growth Condition
                    if max_current < max(target_mass_x1, x2):
                        # Store 'mass' as the label instead of a name
                        graph[current_node].append((target_node, mass, 'down'))

            # --- Check RIGHT Edge: (x1, x2) -> (x1, x2 + mass) ---
            target_mass_x2 = find_mass(x2 + mass)
            
            if target_mass_x2 is not None:
                target_node = (x1, target_mass_x2)
                
                if target_node in valid_nodes_set:
                    # Growth Condition
                    if max_current < max(x1, target_mass_x2):
                        graph[current_node].append((target_node, mass, 'right'))
                        
    return graph

# --- Example Usage ---

# Example nodes
nodes_input = [(0.0, 0.0), (71.0, 0.0), (0.0, 71.0), (100.0, 0.0), (128.0, 0.0)]

# Input is now just a list of floats
# This could include single AAs (71.0) AND combinations (128.0 could be Gly+Ala)
allowed_masses = [
    71.0,  # Mass of 'A'
    57.0,  # Mass of 'G'
    128.0  # Mass of 'A' + 'G' (Combination)
]

adj_graph = build_adjacency_graph(nodes_input, allowed_masses, threshold=0.1)

# Displaying structure
for node, edges in adj_graph.items():
    if edges:
        print(f"Node {node} connects to:")
        for edge in edges:
            # edge[1] is now the float mass
            print(f"  -> {edge[0]} [Mass: {edge[1]}, Type: {edge[2]}]")

Node (0.0, 0.0) connects to:
  -> (71.0, 0.0) [Mass: 71.0, Type: down]
  -> (0.0, 71.0) [Mass: 71.0, Type: right]
  -> (128.0, 0.0) [Mass: 128.0, Type: down]
Node (71.0, 0.0) connects to:
  -> (128.0, 0.0) [Mass: 57.0, Type: down]


In [11]:
from bisect import bisect_left

# Constants
SOURCE_NODE = (0.0, 18.01056)
SINK_NODE = "Sink"

def construct_graph_nodes(spectrum):
    """
    Constructs the 2D grid nodes.
    Ensures the specific SOURCE_NODE (0, 18.01056) exists.
    """
    # 1. Ensure the specific coordinates for the source exist in the spectrum
    #    so the grid generation creates the node naturally.
    #    (Using a set to avoid duplicates)
    extended_spectrum = set(spectrum)
    extended_spectrum.add(SOURCE_NODE[0])
    extended_spectrum.add(SOURCE_NODE[1])
    sorted_spectrum = sorted(list(extended_spectrum))

    nodes = []
    
    # 2. Generate Grid
    for x1 in sorted_spectrum:
        for x2 in sorted_spectrum:
            # Rule: Remove diagonals (x1 == x2), unless it is the Source Node
            # (Note: (0, 18.01) is not diagonal, so it survives naturally, 
            #  but we keep the check just in case you revert to (0,0))
            if x1 != x2 or (x1 == SOURCE_NODE[0] and x2 == SOURCE_NODE[1]):
                nodes.append((x1, x2))
                
    return nodes

def build_adjacency_graph(nodes, allowed_masses, total_peptide_mass, threshold=0.01):
    """
    Constructs the graph edges, including Down, Right, and Sink connections.
    
    Args:
        nodes (list): List of (x1, x2) tuples.
        allowed_masses (list): List of valid mass gaps (AAs).
        total_peptide_mass (float): The total mass M of the peptide.
        threshold (float): Tolerance.
    """
    # Initialize graph
    graph = {node: [] for node in nodes}
    
    # Pre-processing for O(1) lookups
    valid_nodes_set = set(nodes)
    
    # Create a sorted lookup for allowed masses (for Sink check)
    sorted_allowed_masses = sorted(allowed_masses)
    
    # Create a sorted lookup for node coordinate existence (for Down/Right checks)
    # We combine all x1 and x2 values appearing in nodes to form the 'coordinate space'
    unique_coords = sorted(list(set([n[0] for n in nodes] + [n[1] for n in nodes])))

    def find_match_in_list(target, sorted_list):
        """Helper: Returns the matching value from a sorted list if within threshold."""
        idx = bisect_left(sorted_list, target)
        if idx < len(sorted_list) and abs(sorted_list[idx] - target) <= threshold:
            return sorted_list[idx]
        if idx > 0 and abs(sorted_list[idx - 1] - target) <= threshold:
            return sorted_list[idx - 1]
        return None

    for current_node in graph:
        x1, x2 = current_node
        max_current = max(x1, x2)
        
        # ---------------------------------------------------
        # 1. Check Sink Connection
        # Condition: TotalMass - x1 - x2 = Valid Mass
        # ---------------------------------------------------
        remainder = total_peptide_mass - x1 - x2
        
        # Only check if remainder is positive and physically possible
        if remainder > 0: 
            matched_mass = find_match_in_list(remainder, sorted_allowed_masses)
            if matched_mass is not None:
                # We found a bridge to the sink!
                # Edge Type: 'sink', Label: The mass that bridges the gap
                graph[current_node].append((SINK_NODE, matched_mass, 'sink'))

        # ---------------------------------------------------
        # 2. Check Standard Growth Edges (Down / Right)
        # ---------------------------------------------------
        for mass in allowed_masses:
            
            # --- DOWN Edge (x1 -> x1 + mass) ---
            target_x1 = find_match_in_list(x1 + mass, unique_coords)
            if target_x1 is not None:
                target_node = (target_x1, x2)
                if target_node in valid_nodes_set:
                    # Growth Condition
                    if max_current < max(target_x1, x2):
                        graph[current_node].append((target_node, mass, 'down'))

            # --- RIGHT Edge (x2 -> x2 + mass) ---
            target_x2 = find_match_in_list(x2 + mass, unique_coords)
            if target_x2 is not None:
                target_node = (x1, target_x2)
                if target_node in valid_nodes_set:
                    # Growth Condition
                    if max_current < max(x1, target_x2):
                        graph[current_node].append((target_node, mass, 'right'))

    return graph

# --- Example Workflow ---

# 1. Inputs
raw_spectrum = [0.0, 71.03, 18.01, 100.0, 118.0] # Some dummy peaks
aa_masses = [71.03, 57.02] # Ala, Gly
PEPTIDE_MASS = 146.06 # Hypothetical total mass

# 2. Generate Nodes
nodes = construct_graph_nodes(raw_spectrum)

# 3. Build Edges (including Sink)
graph = build_adjacency_graph(nodes, aa_masses, PEPTIDE_MASS, threshold=0.05)

# 4. Inspect specifically the source and potential sink connections
print(f"Graph Keys: {len(graph)} nodes")

if SOURCE_NODE in graph:
    print(f"\nEdges from Source {SOURCE_NODE}:")
    for edge in graph[SOURCE_NODE]:
        print(f"  -> {edge[0]} [Mass: {edge[1]}, Type: {edge[2]}]")

print("\nNodes connecting to Sink:")
found_sink = False
for node, edges in graph.items():
    for edge in edges:
        if edge[0] == SINK_NODE:
            found_sink = True
            print(f"  From {node} -> Sink [Gap Mass: {edge[1]}]")
            
if not found_sink:
    print("  (No nodes connected to sink with these inputs)")

Graph Keys: 30 nodes

Edges from Source (0.0, 18.01056):
  -> (71.03, 18.01056) [Mass: 71.03, Type: down]

Nodes connecting to Sink:
  From (18.01, 71.03) -> Sink [Gap Mass: 57.02]
  From (18.01056, 71.03) -> Sink [Gap Mass: 57.02]
  From (71.03, 18.01) -> Sink [Gap Mass: 57.02]
  From (71.03, 18.01056) -> Sink [Gap Mass: 57.02]


In [92]:
def find_longest_path(graph, start_node, sink_node="Sink"):
    """
    Finds the longest path from start_node to sink_node in the DAG.
    
    Args:
        graph (dict): Adjacency dict {node: [(neighbor, mass, type), ...]}.
        start_node (tuple): The (0.0, 18.01...) source node.
        sink_node (str): The identifier for the sink.
        
    Returns:
        tuple: (max_length, path_of_masses, path_of_nodes)
               - max_length: Number of edges (amino acids).
               - path_of_masses: List of masses (e.g., [71.0, 57.0...]).
               - path_of_nodes: The sequence of graph nodes visited.
    """
    
    # Memoization table: stores result for nodes we've already solved
    # Format: { node: (length, [mass_sequence], [node_sequence]) }
    memo = {}

    def dfs(current_node):
        # 1. Base Case: Reached the Sink
        if current_node == sink_node:
            return (0, [], [sink_node])
        
        # 2. Check Memoization
        if current_node in memo:
            return memo[current_node]
        
        # 3. Recursive Step: Explore all neighbors
        best_length = -1
        best_mass_seq = []
        best_node_seq = []
        
        # If node has no outgoing edges (and isn't sink), it's a dead end.
        if current_node not in graph or not graph[current_node]:
            memo[current_node] = (-1, [], [])
            return (-1, [], [])

        for neighbor, mass, edge_type in graph[current_node]:
            # Recursively solve for the neighbor
            length, mass_seq, node_seq = dfs(neighbor)
            
            # If the neighbor leads to the sink (valid path found)
            if length != -1:
                # We add +1 for the current edge we are traversing
                current_length = 1 + length
                
                # Update if this path is longer than what we've found so far
                if current_length > best_length:
                    best_length = current_length
                    best_mass_seq = [mass] + mass_seq
                    best_node_seq = [current_node] + node_seq
        
        # 4. Store result and return
        if best_length != -1:
            memo[current_node] = (best_length, best_mass_seq, best_node_seq)
        else:
            # Mark as dead end if no neighbors lead to sink
            memo[current_node] = (-1, [], [])
            
        return memo[current_node]

    # Start the search
    return dfs(start_node)

# --- Example Integration with Previous Steps ---

# (Assuming 'graph' and 'SOURCE_NODE' are defined from previous code)
# Example graph structure for demonstration:
# graph = {
#    (0.0, 18.01): [((71.0, 18.01), 71.0, 'down')],
#    (71.0, 18.01): [("Sink", 57.0, 'sink')],
#    "Sink": []
# }
# SOURCE_NODE = (0.0, 18.01)

length, peptide_masses, node_path = find_longest_path(graph, SOURCE_NODE)

if length != -1:
    print(f"Longest Peptide Found (Length {length}):")
    print(f"Sequence of Masses: {peptide_masses}")
    print(f"Path taken: {node_path}")
else:
    print("No valid path from Source to Sink was found.")

No valid path from Source to Sink was found.


In [112]:
pep_seq = 'HADGSFSDEMNTILDNLAARDFINWLIQTKITD'
charge = 4
pep = peptide.Pep(f'[{pep_seq}+{charge}H]{charge}+', end_h20=True)

In [113]:
breaks = [i for i in range(1, len(pep_seq))]
spec = [pep.ion_mass(f'b{i}') for i in breaks] + [pep.ion_mass(f'y{len(pep_seq) - i}') for i in breaks]
spec = [i-1 for i in spec]
spec = [0] + [18.01056] + spec + [pep.pep_mass]
spec.sort()
sorted_array = spec
mid_point = len(sorted_array) // 2 + 2
lower_half = sorted_array[:mid_point]
upper_half = sorted_array[mid_point:]
lower_half.sort()
lower_half

[0,
 18.01056,
 133.04475,
 137.06616,
 208.10327,
 234.09242999999998,
 323.13021000000003,
 347.17649,
 380.15167,
 467.18370000000004,
 475.27145,
 576.3191300000001,
 614.25211,
 701.28414,
 704.3777100000001,
 816.31108,
 817.4617700000001,
 930.5458300000001,
 945.35367,
 1076.39416,
 1116.62514,
 1190.4370900000001,
 1230.6680700000002,
 1291.48477,
 1343.75213,
 1404.56883,
 1490.8205400000002,
 1517.6528899999998,
 1605.8474800000001,
 1632.6798299999998,
 1746.7227599999999,
 1761.9485900000002,
 1832.9857000000002,
 1859.8068199999998,
 1904.0228100000002]

In [115]:
graph = construct_graph_nodes(lower_half)
graph = build_adjacency_graph(nodes = graph, allowed_masses=list(amino_acid_masses.values()), total_peptide_mass=pep.pep_mass - charge*1.007276, threshold=0.05)
length, peptide_masses, node_path = find_longest_path(graph=graph, start_node=SOURCE_NODE)

In [116]:
node_path

[(0.0, 18.01056),
 (0, 133.04475),
 (137.06616, 133.04475),
 (208.10327, 133.04475),
 (208.10327, 234.09242999999998),
 (323.13021000000003, 234.09242999999998),
 (323.13021000000003, 347.17649),
 (380.15167, 347.17649),
 (467.18370000000004, 347.17649),
 (467.18370000000004, 475.27145),
 (467.18370000000004, 576.3191300000001),
 (614.25211, 576.3191300000001),
 (701.28414, 576.3191300000001),
 (701.28414, 704.3777100000001),
 (816.31108, 704.3777100000001),
 (816.31108, 817.4617700000001),
 (816.31108, 930.5458300000001),
 (945.35367, 930.5458300000001),
 (1076.39416, 930.5458300000001),
 (1076.39416, 1116.62514),
 (1190.4370900000001, 1116.62514),
 (1190.4370900000001, 1230.6680700000002),
 (1291.48477, 1230.6680700000002),
 (1291.48477, 1343.75213),
 (1404.56883, 1343.75213),
 (1404.56883, 1490.8205400000002),
 (1517.6528899999998, 1490.8205400000002),
 (1517.6528899999998, 1605.8474800000001),
 (1632.6798299999998, 1605.8474800000001),
 (1746.7227599999999, 1605.8474800000001),
 (1

In [89]:
spec = [pep.ion_mass(f'b{i}') for i in breaks] + [pep.ion_mass(f'y{len(pep_seq) - i}') for i in breaks]
spec

[138.06616,
 209.10327,
 324.13021000000003,
 381.15167,
 349.13534999999996,
 278.09824,
 163.0713,
 106.04984]

In [45]:
pep.pep_mass

635.2771700000001

In [51]:
138.06616 + 496.20375999999993

634.26992

In [50]:
425.16664999999995 + 209.10327

634.26992

In [63]:
635.2771700000001 - (137.06616 + 309.13971)

189.07130000000006

In [74]:
323.13021000000003 +  309.13971

632.26992

In [107]:
pep.pep_mass - charge*1.007276

632.2553160000001

In [123]:
array = 'TD'
sum([amino_acid_masses[i] for i in array])


216.07461999999998

In [124]:
892.6370 * 19

16960.103

In [125]:
len('GLSDGEWQQVLNVWGKVEADIAGHGQEVLIRLFTGHPETLEKFDKFKHLKTEAEMKASEDLKKHGTVVLTALGGILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISDAIIHVLHSKHPGDFGADAQGAMTKALELFRNDIAAKYKELGFQG')

153

In [127]:
pep = peptide.Pep('[GLSDGEWQQVLNVWGKVEADIAGHGQEVLIRLFTGHPETLEKFDKFKHLKTEAEMKASEDLKKHGTVVLTALGGILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISDAIIHVLHSKHPGDFGADAQGAMTKALELFRNDIAAKYKELGFQG+19H]19+')

In [128]:
pep.pep_mass

16960.102369999986

In [129]:
941.96162 * 4

3767.84648

In [130]:
667.90419 * 6

4007.42514